1) import re
파이썬에서 정규 표현식을 사용하기 위해서 re를 import 합니다. 
re에는 찾는 기능을 하는 2종류의 함수가 있는데 match()는 대상 문자열의 처음부터 찾기 시작하고, search()는 대상 문자열의 임의의 위치에 원하는 문자열이 있으면 찾을 수 있습니다.

re lib에 대한 자세한 내용은 https://docs.python.org/3.6/library/re.html?highlight=re#module-re와 https://docs.python.org/3.6/howto/regex.html#regex-howto 를 참조하세요.

2) <body></body>의 내용만 잘라내기
<body>는 HTML 문서의 중간에 위치하니 여기서는 search() 메소드를 사용합니다.

In [3]:
import re

html = "<html><head>some header information</head> \
<Body>it's start. <script src='..'>some script</script> \
<!-- some comments -->some <b>body</b> contents.. \n<a href='some link'>gogo</a> \
그리고 다른 것들.. \
<script>another</script></Body></html>"

body = re.search('<body.*/body>', html, re.I|re.S)

if (body is None) :
    print ("No <body> in html")
    exit()
body = body.group()
print (body)

<Body>it's start. <script src='..'>some script</script> <!-- some comments -->some <b>body</b> contents.. 
<a href='some link'>gogo</a> 그리고 다른 것들.. <script>another</script></Body>


search() 메소드의 첫번째 인수 '<body.*</body>' 는 <body로 시작하고 어떤 글자가(.) 없을 수도 여러개 있을수도 (*) 있고 그 이후에 </body>로 끝난다는 정규 표현식입니다. 

re.I는 re.IGNORECASE와 동일한데 대소문자를 구분하지 않겠다는 의미입니다. 
re.S는 re.DOTALL과 동일한데 의미는 여러 줄에 걸쳐서 정규 표현식을 찾으라는 의미입니다.
.(dot)의 범위가 모든 문자(\n)를 포함하라는 의미입니다.

검색 결과가 없으면 프로그램을 종료하고, 그렇지 않으면 정규 표현식에 부합하는 문자열(body.group())을 다시 body에 넣습니다.


3) <body> 안의 <script>...</script> 삭제
대부분의 <script>는 <head>내에 위치하지만, <body> 내에도 존재할 수 있습니다. 이 <script> 태그는 다른 태그와 달리 <script>와 </script> 사이의 내용도 지우는 것이 컨텐츠만 남기는 목적으로는 맞다고 생각합니다.

문자열 내의 정규 표현식에 부합되는 내용을 지우기 위해서는 sub() 함수를 사용합니다. 
'<script.*?>.*?</script>'의 의미는 <script로 시작하고 그 뒤에 어떤 글자가 없을 수도 있고 여러개 있을 수도 있으며 (*? - ex, src='...'), 그 뒤에는 >가 나오고 다시 어떤 내용이 있을 수 있으며 끝은 </script>로 끝난다는 내용입니다. 

여기서 .* 와 .*?의 차이는 Greedy 하게 탐색을 하느냐 안 하느냐 입니다. 즉, .*는 조건에 가능한 많이 (Greedy) 매칭을 시키고, .*?는 조건에 가능한 적게 매칭을 시킵니다. 
만약 <script.*? 부분을 <script.* 까지만 사용하면 .*의 조건을 가능한 크게 매칭시키기 때문에 첫번째 <script 부터 두번째 </script>를 만날 때까지 .*로 매칭시킬 수 있습니다.

?(non-greedy qualifier)는 *뿐만 아니라 +, ?, {m,n}에도 적용하여 *?, +?, ??, {m,n}? 로 사용할 수 있습니다.

In [4]:
body = re.sub('<script.*?>.*?</script>', '', body, 0, re.I|re.S)
print (body)

<Body>it's start.  <!-- some comments -->some <b>body</b> contents.. 
<a href='some link'>gogo</a> 그리고 다른 것들.. </Body>


3번째 인수 body는 정규 표현식을 적용할 문자열이고, 4번째 0은 부합되는 모든 문자열을 지우라는 의미입니다. 
1을 적게 되면 첫번째 <script>만 삭제됩니다. 
기본으로 안적으면 모든 것을 지우게 되지만, 마지막 인수에서 다시 대소문자 구분을 하지 않고, 여러 줄 대상으로 검색을 해야 하기 때문에 0을 명시해 줬습니다.

실행 결과는 다음과 같습니다. 이제 <script> 태그와 그 내용이 지워진 것을 확인할 수 있습니다.

4) 태그 및 주석 삭제
태그들은 전부 <로 시작하고 >로 끝나며 이것은 주석도 마찬가지입니다. 이번에는 이들 태그와 주석을 지워보도록 하겠습니다.
동일하게 sub() 함수를 사용하며 '<.+?>'는 <로 시작하고 뭔가가 한글자 이상 있으며(.+?) 그 뒤에 >로 끝나는 것의 의미입니다.

In [5]:
text = re.sub('<.+?>', '', body, 0, re.I|re.S)
print (text)

it's start.  some body contents.. 
gogo 그리고 다른 것들.. 


5) 공백 제거
만약에 모든 공백까지 제거하려면 다음처럼 한번 더 sub() 함수를 호출해 주면 됩니다. &nbsp;나 스페이스, 탭(\t), 엔터(유닉스 : \r, 윈도우즈 : \n)에 대해 삭제하는 기능을 하니 필요하면 적용하면 됩니다.

In [6]:
nospace = re.sub('&nbsp;| |\t|\r|\n', '', text)
print (nospace)

it'sstart.somebodycontents..gogo그리고다른것들..


6) 글자수 세기
원래 html과 text, nospace의 각각의 글자수를 세어보도록 하겠습니다.

In [7]:
print ('html = ', len(html), ', text = ', len(text), ', nospace = ', len(nospace))

html =  229 , text =  52 , nospace =  41


7) 특정 URL의 HTML을 가져오기
URL의 내용을 읽어오기 위해서는 urllib.request lib를 사용합니다. 해당 라이브러리의 urlopen()을 이용하여 원하는 주소의 HTML을 가져옵니다. 이후 response.read()로 읽은 후 반드시 utf-8로 decode를 해주도록 합니다. 만약 str(response.read())로 하게 되면 대부분 제대로 된 HTML은 utf-8로 인코딩이 되어있지만 str() 메소드에서는 utf-8로 디코딩을 하지 않아서 한글이 깨지는 문제가 발생하게 됩니다.

In [8]:
import urllib.request

path = 'http://zeany.net/'

with urllib.request.urlopen(path) as response:
	html = response.read().decode("utf-8")

print (html)

<!DOCTYPE html>
<html lang="ko">
<head>
<link rel="stylesheet" type="text/css" href="https://t1.daumcdn.net/tistory_admin/lib/lightbox/css/lightbox.min.css" /><link rel="stylesheet" type="text/css" href="https://t1.daumcdn.net/tistory_admin/assets/blog/7f6d29e1ed95b8cef774f4bf5b47cb2de6f0a5b9/blogs/style/content/font.css?_version_=7f6d29e1ed95b8cef774f4bf5b47cb2de6f0a5b9" /><link rel="stylesheet" type="text/css" href="https://t1.daumcdn.net/tistory_admin/assets/blog/7f6d29e1ed95b8cef774f4bf5b47cb2de6f0a5b9/blogs/style/content/content.css?_version_=7f6d29e1ed95b8cef774f4bf5b47cb2de6f0a5b9" /><!--[if lt IE 9]><script src="https://t1.daumcdn.net/tistory_admin/lib/jquery/jquery-1.12.4.min.js"></script><![endif]--><!--[if gte IE 9]>
<!--><script src="https://t1.daumcdn.net/tistory_admin/lib/jquery/jquery-3.2.1.min.js"></script><!--<![endif]-->
<script src="https://t1.daumcdn.net/tistory_admin/lib/lightbox/js/lightbox-plus-jquery.min.js"></script>
<script>
lightbox.options.fadeDuration = 200

8) 인수로 URL 받기
path를 소스내에 정의하지 말고 인수로 받도록 수정해 보겠습니다. sys 라이브러리의 argv에 인수들은 들어가며 이 때 첫번째 값(argv[0])은 프로그램 이름이고, 두번째 값(argv[1])이 입력할 URL입니다. 만약 2개의 인수를 입력하지 않았다면 잘못된 것이니 프로그램을 종료하고, 정상적으라면 path에 argv[1]를 복사합니다.

In [1]:
import sys

argc = len(sys.argv)

if argc != 2:
	print('usage: python count.py url')
	exit()

path = sys.argv[1]

usage: python count.py url


9) 최종 소스
사용자에게 URL을 입력 받아 해당 HTML의 텍스트의 글자수를 세는 프로그램은 다음과 같습니다.

In [ ]:
import urllib.request
import sys
import re

argc = len(sys.argv)

if argc != 2:
	print('usage: python count.py url')
	exit()

path = sys.argv[1]

with urllib.request.urlopen(path) as response:
	html = response.read().decode("utf-8")

body = re.search('<body.*</body>', html, re.I|re.S)

if (body is None) :
	print ("No <body> in html")
	exit()

body = body.group()
body = re.sub('<script.*?>.*?</script>', '', body, 0, re.I|re.S)

text = re.sub('<.+?>', '', body, 0, re.I|re.S)

nospace = re.sub('&nbsp;| |\t|\r|\n', '', text)

print (nospace)
print ('html = ', len(html), ', text = ', len(text), ', nospace = ', len(nospace))

원하는대로 HTML 문서에서 <body> 내의 모든 태그 및 주석들을 제거한 내용 및 그 글자수입니다.

Reference: https://pypi.org/project/html2text/

In [3]:
import html2text

In [6]:
print(html2text.html2text("<p><strong>Zed's</strong> dead baby, <em>Zed's</em> dead.</p>"))

**Zed's** dead baby, _Zed's_ dead.




In [4]:
h = html2text.HTML2Text()

In [8]:
# Ignore converting links from HTML
h.ignore_links=True
print(h.handle("<p>Hello, <a href='https://www.google.com/earth/'>world</a>!"))

Hello, world!




In [11]:
# Don't Ignore links anymore, I like links
h.ignore_links = False
print(h.handle("<p>Hello, <a href='https://www.google.com/earth/'>world</a>!"))



Hello, [world](https://www.google.com/earth/)!




Reference:http://zeroplus1.zc.bz/jh/web/main.php?id=132&category=ETC